In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import time
from itertools import product
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import matplotlib.pyplot as plt


In [15]:
def train_evaluate_rf(data, target_col='avgPrice', time_series=True):
	"""RandomForest 모델의 그리드서치 수행 및 결과 시각화"""
	# 데이터 준비
	X = data.drop(['date', 'item', target_col], axis=1)
	y = data[target_col]

	# RandomForest 하이퍼파라미터 그리드 정의
	rf_params = {
		'n_estimators': [100, 200, 300],                  # 앙상블을 구성하는 결정 트리의 개수
		'max_depth': [10, 20, 30],                        # 각 트리의 최대 깊이
		'min_samples_split': [2, 5, 10],                  # 내부 노드 분할에 필요한 최소 샘플 수
		'min_samples_leaf': [1, 2, 4],                    # 리프 노드가 되기 위한 최소 샘플 수
		'max_features': ['sqrt', 'log2', 0.8],            # 각 분할에서 고려할 특성의 수
		'bootstrap': [True, False],                       # 트리 생성 시 부트스트랩 샘플링 사용 여부
		'random_state': [1030],                             # 재현성을 위한 랜덤 시드
		'min_impurity_decrease': [0.0, 0.1, 0.2],        # 분할을 위한 최소 불순도 감소량
		'oob_score': [True, False],                       # Out-of-Bag 샘플을 이용한 성능 평가 여부
		'n_jobs': [-1]                                    # 병렬 처리에 사용할 CPU 코어 수
	}   
 
   
	# 교차검증 설정
	if time_series:
		train_size = len(X)//2
		test_size = len(X)//10
		cv = TimeSeriesSplit(
				n_splits=5,
				test_size=test_size,
				gap=0
	)
	else:
		cv = ShuffleSplit(
				n_splits=5, 
				test_size=0.2, 
				random_state=42
	)

	# 그리드서치 수행
	rf = RandomForestRegressor(random_state=1030, n_jobs=-1)
	grid_search = GridSearchCV(
		rf, rf_params, 
		cv=cv,
		scoring='neg_root_mean_squared_error',
		n_jobs=-1,
		verbose=1
	)
   
	grid_search.fit(X, y)

	# 결과 출력
	print("\n** Model Performance **")
	print(f"Best RMSE: {-grid_search.best_score_:.4f}")
	print("\nBest Parameters:")
	for param, value in grid_search.best_params_.items():
		print(f"{param}: {value}")

	# 예측 및 시각화
	y_pred = grid_search.predict(X)

	plt.figure(figsize=(15, 7))
	plt.plot(data['date'], y, label='Actual', color='blue')
	plt.plot(data['date'], y_pred, label='Predicted', color='red', linestyle='--')
	plt.title('Actual vs Predicted Price')
	plt.xlabel('Date')
	plt.ylabel('Price')
	plt.legend()
	plt.grid(True)
	plt.xticks(rotation=45)
	plt.tight_layout()
	plt.show()

	# 평가 지표 계산
	metrics = calculate_metrics(y, y_pred, 0)
	print("\nFinal Metrics:")
	for k, v in metrics.items():
		if k != 'Training_Time':
			print(f"{k}: {v:.4f}")
           
	return grid_search.best_estimator_

In [16]:
data = pd.read_csv('../../../data/features/final_oneHot/광어_price_features_oneHot.csv')
train_evaluate_rf(data, target_col='avgPrice', time_series=True)

Fitting 5 folds for each of 2916 candidates, totalling 14580 fits


KeyboardInterrupt: 